In [1]:
#Fill these file paths with the locations on your machine. 
PATH_TO_CODE = 'C:/Users/blake/OneDrive/Desktop/Computer Vision/Project/pastis-benchmark/code' # path to the code folder of the repo
PATH_TO_PASTIS = 'D:/PASTIS'


import sys
sys.path.append(PATH_TO_CODE)

# import the necessary packages
from torch.nn import ConvTranspose2d
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import Module
from torch.nn import ModuleList
from torch.nn import ReLU
from torchvision.transforms import CenterCrop
from torch.nn import functional as F
from torch.nn import CrossEntropyLoss
import torch
import torch.utils.data as data
import torchnet as tnt

from torchmetrics import JaccardIndex

import numpy as np
from matplotlib.colors import ListedColormap
import matplotlib

import tensorflow as tf

import json
import os
import pickle as pkl
import time

cm = matplotlib.colormaps.get_cmap('tab20')
def_colors = cm.colors
cus_colors = ['k'] + [def_colors[i] for i in range(1,20)]+['w']
cmap = ListedColormap(colors = cus_colors, name='agri',N=21)

def get_rgb(x, batch_index=0, t_show=1):
    """Utility function to get a displayable rgb image 
    from a Sentinel-2 time series.
    """
    im = x['S2'][batch_index, t_show, [2,1,0]].cuda().numpy()
    mx = im.max(axis=(1,2))
    mi = im.min(axis=(1,2))   
    im = (im - mi[:,None,None])/(mx - mi)[:,None,None]
    im = im.swapaxes(0,2).swapaxes(0,1)
    im = np.clip(im, a_max=1, a_min=0)
    return im

def get_radar(x, batch_index=0, t_show=6, orbit='D'):
    """Utility function to get a displayable image 
    from a Sentinel-1 time series.
    """
    im = x['S1{}'.format(orbit)][batch_index, t_show].cuda().numpy()
    mx = im.max(axis=(1,2))
    mi = im.min(axis=(1,2))   
    im = (im - mi[:,None,None])/(mx - mi)[:,None,None]
    im = im.swapaxes(0,2).swapaxes(0,1)
    im = np.clip(im, a_max=1, a_min=0)
    return im

In [2]:
from dataloader import PASTIS_Dataset
from collate import pad_collate

dt = PASTIS_Dataset(PATH_TO_PASTIS, norm=True, target='semantic')
# If you only need to evaluate semantic segmentation use target='semantic'

dl = torch.utils.data.DataLoader(dt, batch_size=100, collate_fn=pad_collate, shuffle=True)

Reading patch metadata . . .
Done.
Dataset ready.


In [3]:
def augment(input_image, input_mask):
   if tf.random.uniform(()) > 0.5:
       # Random flipping of the image and mask
       input_image = tf.image.flip_left_right(input_image)
       input_mask = tf.image.flip_left_right(input_mask)

   return input_image, input_mask

def normalize(input_image, input_mask):
   input_image = tf.cast(input_image, tf.float32) / 255.0
   input_mask -= 1
   return input_image, input_mask

def load_image_train(x, y):
   input_image = x
   input_mask = y
   input_image, input_mask = augment(input_image, input_mask)
   input_image, input_mask = normalize(input_image, input_mask)
   return input_image, input_mask

def load_image_test(x, y):
   input_image = x
   input_mask = y
   input_image, input_mask = normalize(input_image, input_mask)
   return input_image, input_mask

In [8]:
def recursive_todevice(x, device):
    if isinstance(x, torch.Tensor):
        return x.to(device)
    elif isinstance(x, dict):
        return {k: recursive_todevice(v, device) for k, v in x.items()}
    else:
        return [recursive_todevice(c, device) for c in x]


def prepare_output():
    os.makedirs('output', exist_ok=True)
    for fold in range(1, 6):
        os.makedirs(os.path.join("Fold_{}".format(fold)), exist_ok=True)


def checkpoint(fold, log):
    with open(
        os.path.join('output', "Fold_{}".format(fold), "trainlog.json"), "w"
    ) as outfile:
        json.dump(log, outfile, indent=4)


def save_results(fold, metrics):
    if isinstance(metrics, torch.Tensor):
        metrics = metrics.cpu().numpy()
    with open(
        os.path.join('output', "Fold_{}".format(fold), "test_metrics.json"), "w"
    ) as outfile:
        json.dump(metrics, outfile, indent=4)
    
def iterate(
    model, data_loader, criterion, optimizer=None, mode="train", device=None
):
    loss_meter = tnt.meter.AverageValueMeter()
    IoU = JaccardIndex(task='multiclass', num_classes=20).to(device)
    t_start = time.time()

    for i, batch in enumerate(data_loader):
        if device is not None:
            batch = recursive_todevice(batch, device)
        (x, dates), y = batch
        y = y.long()
        x = x['S2'][:, 0, range(0,3), :, :].to(device)

        if mode != "train":
            with torch.no_grad():
                out = model(x)
        else:
            optimizer.zero_grad()
            out = model(x)

        loss = criterion(out, y)
        if mode == "train":
            loss.backward()
            optimizer.step()

        with torch.no_grad():
            pred = out.argmax(dim=1)
            pred = pred.resize(30, 128, 128)

        loss_meter.add(loss.item())

        if (i + 1) % 5 == 0:
            acc = IoU(pred, y)
            print(
                "Step [{}/{}], Loss: {:.4f}, mIoU {:.2f}".format(
                    i + 1, len(data_loader), loss_meter.value()[0], acc
                )
            )

    t_end = time.time()
    total_time = t_end - t_start
    print("Epoch time : {:.1f}s".format(total_time))
    metrics = {
        "{}_loss".format(mode): loss_meter.value()[0],
        "{}_IoU".format(mode): acc.item(),
        "{}_epoch_time".format(mode): total_time,
    }

    return metrics


In [5]:
class Block(Module):
	def __init__(self, inChannels, outChannels):
		super().__init__()
		# store the convolution and RELU layers
		self.conv1 = Conv2d(inChannels, outChannels, 3)
		self.relu = ReLU()
		self.conv2 = Conv2d(outChannels, outChannels, 3)
	def forward(self, x):
		# apply CONV => RELU => CONV block to the inputs and return it
		return self.conv2(self.relu(self.conv1(x)))
	
class Encoder(Module):
	def __init__(self, channels=(3, 16, 32, 64)):
		super().__init__()
		# store the encoder blocks and maxpooling layer
		self.encBlocks = ModuleList(
			[Block(channels[i], channels[i + 1])
			 	for i in range(len(channels) - 1)])
		self.pool = MaxPool2d(2)
		
	def forward(self, x):
		# initialize an empty list to store the intermediate outputs
		blockOutputs = []
		# loop through the encoder blocks
		for block in self.encBlocks:
			# pass the inputs through the current encoder block, store
			# the outputs, and then apply maxpooling on the output
			x = block(x)
			blockOutputs.append(x)
			x = self.pool(x)
		# return the list containing the intermediate outputs
		return blockOutputs
	
class Decoder(Module):
	def __init__(self, channels=(64, 32, 16)):
		super().__init__()
		# initialize the number of channels, upsampler blocks, and
		# decoder blocks
		self.channels = channels
		self.upconvs = ModuleList(
			[ConvTranspose2d(channels[i], channels[i + 1], 2, 2)
			 	for i in range(len(channels) - 1)])
		self.dec_blocks = ModuleList(
			[Block(channels[i], channels[i + 1])
			 	for i in range(len(channels) - 1)])
	def forward(self, x, encFeatures):
		# loop through the number of channels
		for i in range(len(self.channels) - 1):
			# pass the inputs through the upsampler blocks
			x = self.upconvs[i](x)
			# crop the current features from the encoder blocks,
			# concatenate them with the current upsampled features,
			# and pass the concatenated output through the current
			# decoder block
			encFeat = self.crop(encFeatures[i], x)
			x = torch.cat([x, encFeat], dim=1)
			x = self.dec_blocks[i](x)
		# return the final decoder output
		return x
	def crop(self, encFeatures, x):
		# grab the dimensions of the inputs, and crop the encoder
		# features to match the dimensions
		(_, _, H, W) = x.shape
		encFeatures = CenterCrop([H, W])(encFeatures)
		# return the cropped features
		return encFeatures
	
class UNet(Module):
	def __init__(self, encChannels=(3, 16, 32, 64),
		 decChannels=(64, 32, 16),
		 nbClasses=20, retainDim=True,
		 outSize=(128,  128)):
		super().__init__()
		# initialize the encoder and decoder
		self.encoder = Encoder(encChannels)
		self.decoder = Decoder(decChannels)
		# initialize the regression head and store the class variables
		self.head = Conv2d(decChannels[-1], nbClasses, 1)
		self.retainDim = retainDim
		self.outSize = outSize

	def forward(self, x):
		# grab the features from the encoder
		encFeatures = self.encoder(x)
		# pass the encoder features through decoder making sure that
		# their dimensions are suited for concatenation
		decFeatures = self.decoder(encFeatures[::-1][0],
			encFeatures[::-1][1:])
		# pass the decoder features through the regression head to
		# obtain the segmentation mask
		map = self.head(decFeatures)
		# check to see if we are retaining the original output
		# dimensions and if so, then resize the output to match them
		if self.retainDim:
			map = F.interpolate(map, self.outSize)
		# return the segmentation map
		return map

In [6]:
model = UNet().to('cuda:0')
# initialize loss function and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# initialize a dictionary to store training history
H = {"train_loss": [], "test_loss": []}

In [9]:
fold_sequence = [
    [[1, 2, 3], [4], [5]],
    [[2, 3, 4], [5], [1]],
    [[3, 4, 5], [1], [2]],
    [[4, 5, 1], [2], [3]],
    [[5, 1, 2], [3], [4]],
]

np.random.seed(42)
torch.manual_seed(42)
prepare_output()
device = torch.device('cuda:0')
batch_size = 30

for fold, (train_folds, val_fold, test_fold) in enumerate(fold_sequence):
    # Dataset definition
    dt_args = dict(
        folder='D:/PASTIS',
        norm=True,
        target="semantic",
        sats=["S2"],
    )

    dt_train = PASTIS_Dataset(**dt_args, folds=train_folds)
    dt_val = PASTIS_Dataset(**dt_args, folds=val_fold)
    dt_test = PASTIS_Dataset(**dt_args, folds=test_fold)

    collate_fn = pad_collate
    train_loader = data.DataLoader(
        dt_train,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        collate_fn=collate_fn,
    )
    val_loader = data.DataLoader(
        dt_val,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        collate_fn=collate_fn,
    )
    test_loader = data.DataLoader(
        dt_test,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        collate_fn=collate_fn,
    )

    print(
        "Train {}, Val {}, Test {}".format(len(dt_train), len(dt_val), len(dt_test))
    )

    # Training loop
    trainlog = {}
    best_mIoU = 0

    weights = torch.ones(20, device=device).float()
    criterion = CrossEntropyLoss(weight=weights)

    for epoch in range(1, 21):
        print("EPOCH {}/{}".format(epoch, 20))

        train_metrics = iterate(
            model,
            data_loader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
            mode="train",
            device=device,
        )
        if epoch % 4 == 0 and epoch > 3:
            print("Validation . . . ")
            model.eval()
            val_metrics = iterate(
                model,
                data_loader=val_loader,
                criterion=criterion,
                optimizer=optimizer,
                mode="val",
                device=device,
            )

            print(
                "Loss {:.4f}, IoU {:.4f}".format(
                    val_metrics["val_loss"],
                    val_metrics["val_IoU"],
                )
            )

            trainlog[epoch] = {**train_metrics, **val_metrics}
            checkpoint(fold + 1, trainlog)
            if val_metrics["val_IoU"] >= best_mIoU:
                best_mIoU = val_metrics["val_IoU"]
                torch.save(
                    {
                        "epoch": epoch,
                        "state_dict": model.state_dict(),
                        "optimizer": optimizer.state_dict(),
                    },
                    os.path.join(
                        'output', "Fold_{}".format(fold + 1), "model.pth.tar"
                    ),
                )
        else:
            trainlog[epoch] = {**train_metrics}
            checkpoint(fold + 1, trainlog)

    print("Testing best epoch . . .")
    model.load_state_dict(
        torch.load(
            os.path.join(
                'output', "Fold_{}".format(fold + 1), "model.pth.tar"
            )
        )["state_dict"]
    )
    model.eval()

    test_metrics = iterate(
        model,
        data_loader=test_loader,
        criterion=criterion,
        optimizer=optimizer,
        mode="test",
        device=device,
    )
    print(
        "Loss {:.4f},  IoU {:.4f}".format(
            test_metrics["test_loss"],
            test_metrics["test_IoU"]
        )
    )
    save_results(fold + 1, test_metrics)

Reading patch metadata . . .
Done.
Dataset ready.
Reading patch metadata . . .
Done.
Dataset ready.
Reading patch metadata . . .
Done.
Dataset ready.
Train 1455, Val 482, Test 496
EPOCH 1/20


c:\Users\blake\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_tensor.py:775: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


Step [5/48], Loss: 2.7710, mIoU 0.02
Step [10/48], Loss: 2.6057, mIoU 0.02
Step [15/48], Loss: 2.4490, mIoU 0.02
Step [20/48], Loss: 2.3572, mIoU 0.02
Step [25/48], Loss: 2.2963, mIoU 0.02
Step [30/48], Loss: 2.2400, mIoU 0.02
Step [35/48], Loss: 2.2142, mIoU 0.02
Step [40/48], Loss: 2.2013, mIoU 0.02
Step [45/48], Loss: 2.1838, mIoU 0.02
Epoch time : 579.8s
EPOCH 2/20
Step [5/48], Loss: 2.0107, mIoU 0.02
Step [10/48], Loss: 1.9957, mIoU 0.02
Step [15/48], Loss: 1.9977, mIoU 0.02
Step [20/48], Loss: 2.0247, mIoU 0.02
Step [25/48], Loss: 2.0145, mIoU 0.02
Step [30/48], Loss: 2.0005, mIoU 0.02
Step [35/48], Loss: 2.0047, mIoU 0.02
Step [40/48], Loss: 2.0073, mIoU 0.02
Step [45/48], Loss: 2.0159, mIoU 0.02
Epoch time : 504.0s
EPOCH 3/20
Step [5/48], Loss: 2.0403, mIoU 0.02
Step [10/48], Loss: 1.9913, mIoU 0.02
Step [15/48], Loss: 1.9924, mIoU 0.02
Step [20/48], Loss: 2.0029, mIoU 0.02
Step [25/48], Loss: 2.0066, mIoU 0.02
Step [30/48], Loss: 2.0159, mIoU 0.02
Step [35/48], Loss: 2.0093, m